## Обучаем Word2vec: практикум по созданию векторных моделей языка

**Word2vec** - библиотека для получения векторных представлений слов на основе их совместной встречаемости в текстах. Системный Блокъ уже писал ранее о том, как работают эти модели, и вы можете освежить в памяти механизмы работы **word2vec**, прочитав [эту статью](https://vk.com/@sysblok-word2vec-pokazhi-mne-svoi-kontekst-i-ya-skazhu-kto-ty). 

Сейчас мы займемся более практичными и приземленными вещами: научимся использовать **word2vec** в своей повседневной работе. Мы будем использовать реализацию **word2vec** в библиотеке **gensim** для языка программирования **python**.

Тьюториал состоит из двух частей:
* В первой части мы научимся предобрабатывать текстовые файлы и самостоятельно тренировать векторную модель на своих данных.
* Во второй части мы разберёмся, как загружать уже готовые векторные модели и работать с ними. Например, мы научимся выполнять простые операции над векторами слов, такие как «найти слово с наиболее близким вектором» или «вычислить коэффициент близости между двумя векторами слов». Также мы рассмотрим более сложные операции над векторами, например, «найти семантические аналоги» или «найти лишний вектор в группе слов».

Для прохождения тьюториала мы рекомендуем использовать **python3**. Работоспособность кода для **python2** не гарантируется.

## 1. Предобработка текстовых данных и тренировка модели

Перед тем, как переходить к тренировке моделей, бывает необходимо привести данные в формат, удобный для работы. Тексты обычно очищаются от пунктуации, приводятся к нижнему регистру. Этот процесс называется предобработкой текстовых данных. Для того, чтобы начать тренировать векторную модель при помощи **word2vec**, предобработку текста выполнять не обязательно. Для того, чтобы алгоритм смог выучить вектора слов, достаточно разделить текст на предложения. 

Однако в нашем тьюториале мы разберём, как выполнять более глубокую предобработку текста: лемматизацию и частеречный анализ. Это нужно для того, чтобы самим создавать модели, совместимые с уже готовыми [моделями RusVectōrēs](https://rusvectores.org/ru/models/). Ну и просто знание о том, как устроена предобработка текста, может пригодиться во многих задачах обработки языка.

### Обработка текста

Предобработка текстов для тренировки моделей выглядит следующим образом:
* сначала мы приведем все слова к начальной форме (лемматизируем) и удалим стоп-слова;
* затем мы приведем все леммы к нижнему регистру;
* для каждого слова добавим его частеречный тэг.

Давайте попробуем воссоздать процесс предобработки текста на примере рассказа [О. Генри "Русские соболя"](https://rusvectores.org/static/henry_sobolya.txt). Для предобработки можно использовать различные тэггеры, мы сейчас будем использовать [*UDPipe*](https://ufal.mff.cuni.cz/udpipe), чтобы сразу получить частеречную разметку в виде Universal POS-tags. Сначала установим обертку *UDPipe* для Python:

`pip install ufal.udpipe`

*UDPipe* использует предобученные модели для лемматизации и тэггинга. Вы можете использовать [уже готовую модель](https://rusvectores.org/static/models/udpipe_syntagrus.model) или обучить свою. 

Чтобы загружать файлы, можно использовать питоновскую библиотеку **wget**:

`pip install wget`

Кусок кода ниже скачает рассказ О’Генри и модель UDPipe для лингвистической предобработки. Модель весит 40 мегабайт, поэтому ячейка может выполнятся некоторое время, особенно если у вас небыстрый интернет. 

In [1]:
import wget
import sys

udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
text_url = 'https://rusvectores.org/static/henry_sobolya.txt'

modelfile = wget.download(udpipe_url)
textfile = wget.download(text_url)

Приступим к собственно предобработке текста. Попробуем лемматизировать текст и добавить частеречные тэги при помощи этой функции:

In [2]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn


Эту функцию можно также изменить под конкретную задачу. Например, если частеречные тэги нам не нужны, в функции ниже выставим `keep_pos=False`. Если необходимо сохранить знаки пунктуации, можно выставить `keep_punct=True`. 

Теперь загружаем модель *UDPipe*, читаем текстовый файл и обрабатываем его при помощи нашей функции. В файле должен содержаться необработанный текст (одно предложение на строку или один абзац на строку).
Этот текст токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход мы получаем последовательность разделенных пробелами лемм с частями речи ("зеленый\_NOUN трамвай\_NOUN").

In [3]:
from ufal.udpipe import Model, Pipeline
import os
import re

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_model_url.split('/')[-1]

    if not os.path.isfile(modelfile):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_model_url)

    print('\nLoading the model...', file=sys.stderr)
    model = Model.load(modelfile)
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    print('Processing input...', file=sys.stderr)
    lines = text.split('\n')
    tagged = []
    for line in lines:
        # line = unify_sym(line.strip()) # здесь могла бы быть ваша функция очистки текста
        output = process(process_pipeline, text=line)
        tagged_line = ' '.join(output)
        tagged.append(tagged_line)
    return '\n'.join(tagged)

In [4]:
text = open(textfile, 'r', encoding='utf-8').read()
processed_text = tag_ud(text=text, modelfile=modelfile)
print(processed_text[:350])
with open('my_text.txt', 'w', encoding='utf-8') as out:
    out.write(processed_text)


Loading the model...
Processing input...


русский_PROPN  соболь_NOUN о.::генри_PROPN 
когда_SCONJ синий_ADJ как_SCONJ ночь_NOUN глаз_NOUN Молли_VERB Мак-Кивер_PROPN  класть_VERB малыш::Брэди_PROPN  на_ADP оба_NUM лопатка_NOUN он_PRON вынужденный_ADJ быть_AUX покидать_VERB ряд_NOUN банда_NOUN «Дымовый_ADJ труба»_NOUN таков_ADJ власть_NOUN нежный_ADJ укор_NOUN подружка_NOUN и_CCONJ она_PRON 


Наша функция напечатает обработанный текст, который мы теперь можем также сохранить в файл. 

Итак, в ходе этой части тьюториала мы научились от "сырого текста" приходить к лемматизированному тексту с частеречными тэгами, который уже можно подавать на вход модели! Теперь теперь попробуем натренировать векторную модель.

### Процесс тренировки модели 

Для работы с эмбеддингами слов существуют различные библиотеки: [gensim](https://radimrehurek.com/gensim/), [keras](https://keras.io/), [tensorflow](https://www.tensorflow.org/), [pytorch](https://pytorch.org/). Мы будем работать с библиотекой *gensim*.


***Gensim***  - изначально библиотека для тематического моделирования текстов. Однако помимо различных алгоритмов для *topic modeling* в ней реализованы на **python** и алгоритмы из тулкита **word2vec** (который в оригинале был написан на C++). Прежде всего, если **gensim** у вас на компьютере не установлен, нужно его установить:

`pip install gensim`

Gensim регулярно обновляется, так что не будет лишним удостовериться, что у вас установлена последняя версия, а при необходимости проапдейтить библиотеку:

`pip install gensim --upgrade` 

или 

`pip install gensim -U`

При подготовке этого тьюториала использовался **gensim** версии 3.7.0.

Поскольку обучение и загрузка моделей могут занимать продолжительное время, иногда бывает полезно вести лог событий. Для этого используется стандартная питоновская библиотека **logging**.

In [5]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

На вход модели мы даем наш обработанный текстовый файл (либо любой другой текст, важно лишь, что каждое предложение должно быть на отдельной строчке).

In [6]:
f = 'my_text.txt'
data = gensim.models.word2vec.LineSentence(f)

Инициализируем модель. Параметры в скобочках:
* data - данные, 
* size - размер вектора, 
* window - размер окна наблюдения,
* min_count - мин. частотность слова в корпусе, которое мы берем,
* sg - используемый алгоритм обучение (0 - CBOW, 1 - Skip-gram))

In [7]:
model = gensim.models.Word2Vec(data, size=500, window=10, min_count=2, sg=0)

2020-10-01 18:00:29,028 : INFO : collecting all words and their counts
2020-10-01 18:00:29,120 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-10-01 18:00:29,174 : INFO : collected 981 word types from a corpus of 2085 raw words and 65 sentences
2020-10-01 18:00:29,175 : INFO : Loading a fresh vocabulary
2020-10-01 18:00:29,178 : INFO : effective_min_count=2 retains 252 unique words (25% of original 981, drops 729)
2020-10-01 18:00:29,179 : INFO : effective_min_count=2 leaves 1356 word corpus (65% of original 2085, drops 729)
2020-10-01 18:00:29,186 : INFO : deleting the raw counts dictionary of 981 items
2020-10-01 18:00:29,188 : INFO : sample=0.001 downsamples 89 most-common words
2020-10-01 18:00:29,189 : INFO : downsampling leaves estimated 801 word corpus (59.1% of prior 1356)
2020-10-01 18:00:29,214 : INFO : estimated required memory for 252 words and 500 dimensions: 1134000 bytes
2020-10-01 18:00:29,215 : INFO : resetting layer weights
2020-10-01 1

Мы создаем модель, в которой размерность векторов — 500, размер окна наблюдения — 10 слов, алгоритм обучения — CBOW, слова, встретившиеся в корпусе только 1 раз, не используются. После тренировки модели можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировать.

In [8]:
model.init_sims(replace=True)

2020-10-01 18:00:32,571 : INFO : precomputing L2-norms of word weight vectors


Смотрим, сколько в модели слов:

In [9]:
print(len(model.wv.vocab))

252


И сохраняем!

In [10]:
model.save('my.model')

2020-10-01 18:00:34,203 : INFO : saving Word2Vec object under my.model, separately None
2020-10-01 18:00:34,206 : INFO : not storing attribute vectors_norm
2020-10-01 18:00:34,208 : INFO : not storing attribute cum_table
2020-10-01 18:00:34,226 : INFO : saved my.model


## 2. Работа с векторными моделями при помощи библиотеки Gensim

Для своих индивидуальных нужд и экспериментов бывает полезно самому натренировать модель на нужных данных и с нужными параметрами. Но для каких-то общих целей модели уже есть как для русского языка, так и для английского.

Модели для русского скачать можно здесь - https://rusvectores.org/ru/models/

Существуют несколько форматов, в которых могут храниться модели. Во-первых, данные могут храниться в нативном формате *word2vec*, при этом модель может быть бинарной или не бинарной. Для загрузки модели в формате *word2vec* в классе `KeyedVectors` (в котором хранится большинство относящихся к дистрибутивным моделям функций) существует функция `load_word2vec_format`, а бинарность модели можно указать в аргументе `binary` (внизу будет пример). Помимо этого, модель можно хранить и в собственном формате *gensim*, для этого существует класс `Word2Vec` с функцией `load`. Поскольку модели бывают разных форматов, то для них написаны разные функции загрузки; бывает полезно учитывать это в своем скрипте. Наш код определяет тип модели по её расширению, но вообще файл с моделью может называться как угодно, жестких ограничений для расширения нет.

Давайте скачаем новейшую модель для русского языка, созданную на основе [Национального корпуса русского языка (НКРЯ)](http://www.ruscorpora.ru/), и загрузим в её в память (поскольку zip-архив с моделью весит почти 500 мегабайт, следующая ячейка выполнится у вас не сразу!). Распаковывать скачанный архив для обычных моделей не нужно, так как его содержимое прочитается при помощи специальной инструкции:

In [11]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/20/180.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

2020-10-01 18:01:02,813 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>
2020-10-01 18:01:07,058 : INFO : loaded (189193, 300) matrix from <zipfile.ZipExtFile [closed]>


Допустим, нам интересны такие слова (пример для русского языка):

In [12]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN', 'студент_ADJ']

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [13]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print(word + ' is not present in the model')

2020-10-01 18:01:12,321 : INFO : precomputing L2-norms of word weight vectors


день_NOUN
неделя_NOUN 0.7375995516777039
день_PROPN 0.7067667245864868
месяц_NOUN 0.7037326693534851
час_NOUN 0.6643949747085571
утро_NOUN 0.6526744961738586
вечер_NOUN 0.6038411855697632
сутки_NOUN 0.5923081040382385
воскресенье_NOUN 0.5842781066894531
полдень_NOUN 0.5743687152862549
суббота_NOUN 0.5345946550369263


ночь_NOUN
ночь_PROPN 0.8310786485671997
вечер_NOUN 0.7183678150177002
рассвет_NOUN 0.696594774723053
ночи_NOUN 0.6920218467712402
полночь_NOUN 0.6704976558685303
ночь_VERB 0.6615264415740967
утро_NOUN 0.6263935565948486
ночной_ADJ 0.6024709343910217
полдень_NOUN 0.5835086107254028
сумерки_NOUN 0.5671443343162537


человек_NOUN
человек_PROPN 0.7850059270858765
человеческий_ADJ 0.5915265679359436
существо_NOUN 0.573693037033081
народ_NOUN 0.5354466438293457
личность_NOUN 0.5296981930732727
человечество_NOUN 0.5282931327819824
человкъ_PROPN 0.5047001838684082
индивидуум_NOUN 0.5000404715538025
нравственный_ADJ 0.4972919821739197
потому_ADV 0.49293625354766846


семантика_NOU

Находим косинусную близость пары слов:

In [14]:
print(model.similarity('человек_NOUN', 'обезьяна_NOUN'))

0.22025342


## 3. Более сложные операции над векторами

Помимо более простых операций над векторами (нахождение косинусной близости между двумя векторами и ближайших соседей вектора) **gensim** позволяет выполнять и более сложные операции над несколькими векторами. Так, например, мы можем найти лишнее слово в группе. Лишним словом является то, вектор которого наиболее удален от других векторов слов.

In [15]:
print(model.doesnt_match('яблоко_NOUN груша_NOUN виноград_NOUN банан_NOUN лимон_NOUN картофель_NOUN'.split()))

картофель_NOUN


/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


Также можно складывать и вычитать вектора нескольких слов. Например, сложив два вектора и вычтя из них третий вектор, мы можем решить своеобразную пропорцию. Подробнее о семантических пропорциях вы можете прочитать в [материале Системного Блока](https://vk.com/@sysblok-vo-chto-prevraschaetsya-zhizn-bez-lubvi).

In [16]:
print(model.most_similar(positive=['пицца_NOUN', 'россия_NOUN'], negative=['италия_NOUN'])[0][0])

гамбургер_NOUN


# Заключение

В этом тьюториале мы постарались разобраться с тем, как работать с семантическими векторными моделями и библиотекой **gensim**. Теперь вы можете:
* осуществлять предобработку текстовых данных, что может пригодиться во многих задачах обработки естественного языка;
* тренировать векторные семантические модели. Формат моделей совместим с моделями, представленными на веб-сервисе **RusVectōrēs**;
* осуществлять простые операции над векторами слов.

Мы надеемся, что этот тьюториал поможет нашим читателям поглубже окунуться в мир дистрибутивной семантики и использовать эти инструменты в своей работе!